# Generating NN Code Bench OOB Extension

This is an experimental process to expand the smaller sized samples of NN Code Bench for the purpose of creating a dataset that can be used to train NN models. The samples that are going to be used to expand the dataset, through fuzzing techniques, are the **Polynomial Approximation Networks**, the **Hopfield Nets**, the **Reach Prob Density**, and the **Reinforcement Learning** samples. The **Activation Functions** and the **Math Functions** will not be used as they are too small or don't have any code that can cause memory leaks if manipulated (Dont use math and activation categories as they dont contain array out of bounds (They are too small)).

The code extensions will focus on array out of bounds, as it is an easy safety property to detect.

* Timeout: 900s as in SVComp - 30s for testing.

The following research questions are going to be answered by the expansion of the dataset:

* What size can ESBMC handle?
* Will ESBMC be able to find the bug in a reasonable time? Compare 30s vs 900s.
* How do other software versifiers perform with the task of solving the expanded dataset?

## TODO

- Remove assert safety properties by Eduardo in the code.


## Preprocess Data

This notebook takes care of processing the data of NN Codebench by compiling them into executables. Takes the programs in `networks` and compiles them to `bin` in the same structure.

### Base Dataset

#### Building Base Source Dataset

Building it from scratch.

In [13]:
import os
import shutil
from contextlib import chdir
from typing import Tuple
from subprocess import Popen, STDOUT, PIPE

In [14]:
def popen(cmd: str) -> Tuple[str, str]:
    process = Popen(cmd, stdout=PIPE, stderr=STDOUT)
    (output_bytes, err_bytes) = process.communicate()
    # Return
    exit_code = process.wait()
    output: str = str(output_bytes).replace("\\n", "\n")
    err: str = str(err_bytes).replace("\\n", "\n")
    return output, err

In [3]:
# Create the build folder in case it does not exist.
BUILD_PATH: str = "./build"
if os.path.exists(BUILD_PATH):
    print(f"{BUILD_PATH} already exists.")
else:
    os.makedirs(BUILD_PATH)
    print(f"Created {BUILD_PATH}")

./build already exists.


In [3]:
# Build all base benchmarks. Note: From my experience, these do not build on an ARM machine but, can be
# transferred from an x86_64 based machine and used, since they are still C source code files and not
# binary.
with chdir(BUILD_PATH):
    print("Building all samples...")
    print(os.popen("cmake ..").read())
    print(os.popen("make -j4 install").read())
    print("Completed...")

Building all samples...
-- The C compiler identification is GNU 10.2.1
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /home/yiannis/git/plain_c_nn_benchmark/build

Scanning dependencies of target stub
Scanning dependencies of target reach_prob_lib
Scanning dependencies of target keras_lib
Scanning dependencies of target reinforcement_lib
[  1%] Building C object networks/CMakeFiles/reach_prob_lib.dir/reach_prob_density/gcas.c.o
[  1%] Building C object includes/CMakeFiles/keras_lib.dir/keras2c/k2c_activations.c.o
[  1%] Building C object includes/CMakeFiles/stub.dir/verifier_stubs.c.o
[  1%] Building C object networks/CMakeFiles/reinforcement_lib.dir/reinforcement_learning/cartpole.c.o
[  1%] Linking C static library libstub.a
[  2%] Building C object networks/C

cc: error: unrecognized command-line option ‘-m32’
make[2]: *** [properties/activation_functions/CMakeFiles/softsign_0_safe.c-amalgamation.dir/build.make:77: properties/activation_functions/CMakeFiles/softsign_0_safe.c-amalgamation] Error 1
make[1]: *** [CMakeFiles/Makefile2:1708: properties/activation_functions/CMakeFiles/softsign_0_safe.c-amalgamation.dir/all] Error 2
make[1]: *** Waiting for unfinished jobs....
cc: error: unrecognized command-line option ‘-m32’
make[2]: *** [properties/activation_functions/CMakeFiles/softmax_1_unsafe.c-amalgamation.dir/build.make:77: properties/activation_functions/CMakeFiles/softmax_1_unsafe.c-amalgamation] Error 1
make[1]: *** [CMakeFiles/Makefile2:1735: properties/activation_functions/CMakeFiles/softmax_1_unsafe.c-amalgamation.dir/all] Error 2
make: *** [Makefile:149: all] Error 2


#### Remove intermediate files (speed up processing).

In [4]:
if os.path.exists(BUILD_PATH):
    count: int = 0
    with chdir(BUILD_PATH):
        files: list[str] = os.listdir(".")
        for file in files:
            if os.path.isfile(file) and file.endswith((".c", ".i", ".yml")):
                os.remove(file)
                count += 1
    print(f"Cleaned up {count} intermediate files.")

Cleaned up 0 intermediate files.


In [5]:
build_folders: list[str] = [
    # f"{BUILD_PATH}/export/hopfield_nets",
    f"{BUILD_PATH}/export/poly_approx",
    # f"{BUILD_PATH}/export/reach_prob_density",
    # f"{BUILD_PATH}/export/reinforcement_learning",
]

#### Extracting it from SV-Bench

They mirror the built ones. So if we want to use the ones from SV-Bench, we can do so by
looking at the manually built files names (for example poly-approx) and transfering the
ones we want from the SV-Bench folder to the built one (replacing the manually built ones).

Why? Because of [this](https://github.com/emanino/plain_c_nn_benchmark/issues/24).

Create a `sv-bench` folder in the root of the project with the contents of `sv-bench/c/neural-networks`.

In [7]:
def transfer_from_sv_bench(dest_path: str, delete_source=False) -> None:
    """Transfers the files from SV Bench to the dest path. If `delete_source` is True
    then the transfer will also remove from the SV_Bench folder."""
    SV_BENCH_FOLDER = "sv-bench"
    print(f"Moving files from {SV_BENCH_FOLDER} to {dest_path}")
    # The folder for SV-Bench neural network samples.
    if not os.path.exists(SV_BENCH_FOLDER):
        raise FileNotFoundError(f"Directory SV-Bench does not exist: {SV_BENCH_FOLDER}")
    if not os.path.exists(dest_path):
        raise FileNotFoundError(f"Directory {dest_path} does not exist.")

    count: int = 0
    for idx, file in enumerate(os.listdir(dest_path)):
        src: str = f"{SV_BENCH_FOLDER}/{file}"
        dst: str = f"{dest_path}/{file}"
        if os.path.isfile(src) and file.endswith(".c"):
            shutil.copy(src=src, dst=dst)
            print(f"{idx}: {file}")
            count += 1

    print(f"Completed moving {count} files from {SV_BENCH_FOLDER} to {dest_path}\n")

# Copy SV-Bench files to each folder used.
for folder in build_folders:
    transfer_from_sv_bench(folder)

Moving files from sv-bench to ./build/export/hopfield_nets
5: tanh_w4_r1_case_1_safe.c-amalgamation.c
7: softsign_w16_r3_case_0_safe.c-amalgamation.c
8: softsign_w64_r1_case_0_unsafe.c-amalgamation.c
9: softsign_w4_r4_case_0_safe.c-amalgamation.c
15: tanh_w64_r4_case_1_safe.c-amalgamation.c
18: tanh_w64_r3_case_0_safe.c-amalgamation.c
24: tanh_w16_r2_case_1_unsafe.c-amalgamation.c
26: softsign_w4_r3_case_0_safe.c-amalgamation.c
32: tanh_w64_r3_case_1_safe.c-amalgamation.c
33: softsign_w8_r2_case_1_unsafe.c-amalgamation.c
34: softsign_w4_r2_case_1_unsafe.c-amalgamation.c
37: tanh_w32_r2_case_0_safe.c-amalgamation.c
41: tanh_w8_r1_case_1_safe.c-amalgamation.c
43: tanh_w8_r3_case_0_safe.c-amalgamation.c
48: softsign_w64_r1_case_1_safe.c-amalgamation.c
50: tanh_w16_r1_case_0_unsafe.c-amalgamation.c
52: tanh_w32_r1_case_0_unsafe.c-amalgamation.c
54: softsign_w16_r2_case_0_safe.c-amalgamation.c
55: softsign_w32_r2_case_0_safe.c-amalgamation.c
56: tanh_w4_r3_case_0_safe.c-amalgamation.c
59: s

### Prepare and Build Base Samples

We now have the base samples built. So we need to mutate and build base samples.

In [22]:
def build_sample(sample_file_path: str) -> None:
    # Mull compiler plugin parameters.
    MULL: str = "-fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line"

    # Check if sample file exists in path specified.
    if not os.path.exists(sample_file_path):
        raise FileExistsError(f"No file found in {sample_file_path}")

    # Get the base name.
    sample_name: str = os.path.basename(sample_file_path)

    # Check if it is a C source code file.
    if not sample_name.endswith(".c"):
        raise ValueError(f"{sample_name} is not a C source code filename.")

    # Check if it exists in the bin folder.
    if not os.path.exists("bin"):
        raise FileNotFoundError("No bin folder...")

    # Verifier stub include file.
    ver_h: str = "includes"
    # Add networks folder as include directory.
    poly_aprox_h: str = "networks"

    # Source files to compile and create binary.
    source_files: list[str] = ["includes/verifier_stubs.c", sample_file_path]
    
    # Intermediate source files, because Mull requires to build each file individually into an .o file.
    # Automatically built from source files list.
    object_files: list[str] = ["bin/intermediate/" + os.path.basename(source_file)[:-2] + ".o" for source_file in source_files]
    
    # Build each file into an object file (-c) and then combine. This is due to the Mull plugin,
    # which only works on single files.
    for source_file, object_file in zip(source_files, object_files):
        cmd: str = f"clang-15 {MULL} -c {source_file} -I{ver_h} -I{poly_aprox_h} -o {object_file}"
        print("\n" + cmd)
        print(os.popen(cmd).read())
    else:
        print()

    out: str = f"bin/poly_approx/{sample_name.removesuffix('.c')}"

    # Link object files into single binary.
    cmd: str = f"clang-15 {MULL} {' '.join(object_files)} -I{ver_h} -I{poly_aprox_h} -lm -o {out}"
    print(cmd)
    print(os.popen(cmd).read())

    print(f"Compilation Complete: {out}")

In [20]:
# Create the bin folder in case it does not exist.
if os.path.exists("bin"):
    print("bin already exists.")
else:
    os.makedirs("bin")
    print("Created bin")

if os.path.exists("bin/intermediate"):
    print("bin/intermediate already exists.")
else:
    os.makedirs("bin/intermediate")
    print("Created bin/intermediate")

bin already exists.
bin/intermediate already exists.


In [23]:
count: int = 0
for folder in build_folders:
    print("Compiling folder:", folder)
    for file in os.listdir(folder):
        file_path: str = f"{folder}/{file}"
        if os.path.isfile(file_path) and file.endswith(".c"):
            print(f"Building {count} {file_path}")
            build_sample(file_path)
            print()
            count += 1
    print()

print(f"Compilation of samples has completed: Total {count}...")
del count

Compiling folder: ./build/export/poly_approx
Building 0 ./build/export/poly_approx/poly_32_32_32_thresh_3_unsafe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c ./build/export/poly_approx/poly_32_32_32_thresh_3_unsafe.c-amalgamation.c -Iincludes -Inetworks -o bin/intermediate/poly_32_32_32_thresh_3_unsafe.c-amalgamation.o


error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_32_32_32_thresh_3_unsafe.c-amalgamation.o'




clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_32_32_32_thresh_3_unsafe.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_32_32_32_thresh_3_unsafe.c-amalgamation

Compilation Complete: bin/poly_approx/poly_32_32_32_thresh_3_unsafe.c-amalgamation

Building 1 ./build/export/poly_approx/poly_64_64_64_thresh_2_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c ./build/export/poly_approx/poly_64_64_64_thresh_2_safe.c-amalgamation.c -Iincludes -Inetworks -o bin/intermediate/poly_64_64_64_thresh_2_safe.c-amalgamation.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/

error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_64_64_64_thresh_2_safe.c-amalgamation.o'
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.



Compilation Complete: bin/poly_approx/poly_64_64_64_thresh_2_safe.c-amalgamation

Building 2 ./build/export/poly_approx/poly_32_32_32_thresh_2_safe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c ./build/export/poly_approx/poly_32_32_32_thresh_2_safe.c-amalgamation.c -Iincludes -Inetworks -o bin/intermediate/poly_32_32_32_thresh_2_safe.c-amalgamation.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_32_32_32_thresh_2_safe.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_32_32_32_thresh_2_safe.c-amalgamation

Compilation Complete: bin/poly_approx/poly_32_32_32_thresh_2_safe.c

...

clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_32_32_thresh_2_safe.c-amalgamation.o'
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_16_16_thresh_2_safe.c-amalgamation.o'
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file 



clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c ./build/export/poly_approx/poly_64_64_thresh_3_unsafe.c-amalgamation.c -Iincludes -Inetworks -o bin/intermediate/poly_64_64_thresh_3_unsafe.c-amalgamation.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_64_64_thresh_3_unsafe.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_64_64_thresh_3_unsafe.c-amalgamation

Compilation Complete: bin/poly_approx/poly_64_64_thresh_3_unsafe.c-amalgamation

Building 97 ./build/export/poly_approx/poly_32_32_thresh_5_unsafe.c-amalgamation.c

clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line -c includes/verifier_stubs.c -Iincludes -Inetworks -o bin/intermediate/verifier_stubs.o


clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir

error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_64_64_thresh_3_unsafe.c-amalgamation.o'
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.
error: unable to load plugin '/usr/lib/mull-ir-frontend-15': 'Could not load library '/usr/lib/mull-ir-frontend-15': /usr/lib/mull-ir-frontend-15: cannot open shared object file: No such file or directory'
1 error generated.




clang-15 -fexperimental-new-pass-manager -fpass-plugin=/usr/lib/mull-ir-frontend-15 -g -grecord-command-line bin/intermediate/verifier_stubs.o bin/intermediate/poly_32_32_thresh_5_unsafe.c-amalgamation.o -Iincludes -Inetworks -lm -o bin/poly_approx/poly_32_32_thresh_5_unsafe.c-amalgamation

Compilation Complete: bin/poly_approx/poly_32_32_thresh_5_unsafe.c-amalgamation


Compilation of samples has completed: Total 98...


clang: error: no such file or directory: 'bin/intermediate/verifier_stubs.o'
clang: error: no such file or directory: 'bin/intermediate/poly_32_32_thresh_5_unsafe.c-amalgamation.o'


#### Delete Intermediate Build Files

In [ ]:
if os.path.exists("./bin/intermediate/"):
    shutil.rmtree("./bin/intermediate/")
    print(f"Cleaned up /bin/intermediate/ intermediate files.")

### Generating Mutation Patches

Now that the mutations have been inserted into the compiled base samples, we have the mutation fuzzed samples. The mutation fuzzed samples can then be **executed** to produce the patches. The _killed_ patches are ones that cause the program execution to defer.

> #### WARNING! 
> Running the following section will execute code on the system. Make sure that the code is safe to run.

In [ ]:
def gen_mutation(sample_name: str) -> None:
    print(os.popen(f"mull-runner-15 --reporters Patches --report-dir patches {sample_name}").read())

In [ ]:
gen_mutation("bin/poly_approx/poly_16_16_16_16_thresh_5_unsafe.c-amalgamation")